In [32]:
import stanza
import sys
import os
import re
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

In [79]:
##load stanza model
language = 'en'
#stanza.download(language, verbose = False)
nlp = stanza.Pipeline(language, processors='tokenize, mwt, ner, lemma, pos, depparse', verbose = False)


In [5]:
#string = open(f'books/{text_name}.txt', 'r', encoding="utf8")
text_name = 'PrideandPrejudice'
with open(f'books/{text_name}.txt', 'r', encoding="utf8") as file:
    data = file.read().replace('\n', '')

In [80]:
### run model on the corpus
doc = nlp(data)

In [201]:
### network version 1: two people are connected if they have the same head in a sentence (are subject-object)

entities_tot = [(ent.text, ent.type, i) for  i, sent in enumerate(doc.sentences) for ent in sent.ents]
people = [(i[0], i[2]) for i in entities_tot if i[1] == 'PERSON']
plp = [i[0] for i in people]
u, count = np.unique(plp, return_counts=True)
count_sort_ind = np.argsort(-count)
people_ordered = u[count_sort_ind]

nw_matrix = np.zeros((len(people_ordered), len(people_ordered)), dtype = int)


for sent in doc.sentences:
    words = [(word.id, word.text, word.head) for word in sent.words]
    entities = [(ent.text, ent.type) for ent in sent.ents]
    names = [i[0] for i in entities if len(np.where(i[0] == people_ordered)[0])>0]
    if len(names) > 0: 
        people = [[word for word in words if name==word[1]] for name in names]
        people = [item for sublist in people for item in sublist]
     
        for j in people:
            for l in people:
                if j[2] == l[2]:
                    indx_j = np.where(j[1] == people_ordered)[0][0]
                    indx_l = np.where(l[1] == people_ordered)[0][0]
                    if indx_j != indx_l:
                        nw_matrix[indx_j][indx_l] += 1
                        nw_matrix[indx_l][indx_j] += 1


In [39]:
### 2: two people are connected if they occur in the same sentence at least once
entities = [(ent.text, ent.type, i) for  i, sent in enumerate(doc.sentences) for ent in sent.ents]
people = [(i[0], i[2]) for i in entities if i[1]== 'PERSON' ]
plp = [i[0] for i in people]
u, count = np.unique(plp, return_counts=True)
count_sort_ind = np.argsort(-count)
people_ordered = u[count_sort_ind]

nw_matrix = np.zeros((len(people_ordered), len(people_ordered)), dtype = int)

plp_snt_matrix = np.zeros((len(people_ordered), len(doc.sentences)), dtype = int)
for i in people:
    person = i[0]
    sentence = i[1]
    person_indx = np.where(person == people_ordered)[0][0]
    plp_snt_matrix[person_indx][sentence] += 1

for j in range(len(doc.sentences)):
    plp_in_snt = [i for i in range(len(people_ordered)) if plp_snt_matrix[i, j]> 0]
    for l in plp_in_snt:
        for k in plp_in_snt:
            if l != k:
                nw_matrix[l][k] += 1
                nw_matrix[k][l] += 1


In [203]:
nw_unweighted = np.sign(nw_matrix)
G = nx.from_numpy_array(nw_unweighted)


In [204]:
lbls = {i:people_ordered[i] for i in range(len(people_ordered))}

In [205]:
Gcc = sorted(nx.connected_components(G), key=len, reverse=True)
G0 = G.subgraph(Gcc[0])

pos = nx.drawing.nx_agraph.graphviz_layout(G0, prog = 'neato')
lbls_subgraph = {n:lab for n,lab in lbls.items() if n in G0.nodes}

In [206]:

plt.figure(dpi=800, figsize = (12,12))

nx.draw_networkx_nodes(G0, pos, node_size = 150)
nx.draw_networkx_edges(G0, pos, width = 0.2, alpha = 0.6)

labels = nx.draw_networkx_labels(G0, pos, labels=lbls_subgraph)